In [1]:
from os import environ as ENV

DATA = ENV['PWD'] + '/data/'

In [2]:
import sys

sys.path.append('../src')

from should_be_stdlib import is_array_lesser
from circuit_postprocess import *
from circuits import (
    circuit_angle_swap,
    circuit_angle_qft_swap,
    circuit_amp_iamp,
    circuit_amp_iamp_qft,
)

In [3]:
from itertools import combinations

import pennylane as qml
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
from qbraid import transpile

/home/user/work/quadrigems/.venv/lib/python3.11/site-packages/qbraid/_entrypoints.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [5]:
tuning_curves_rescaled = pd.read_csv(DATA + 'data_tuning-curves_rescaled.csv', index_col=0)
tuning_curves_rescaled

,3.0,4.2,6.0,8.5,12.0,17.0,24.0,33.9,48.0
23,0.272445,0.226280,0.528095,0.342139,0.478668,0.216327,0.388480,0.282726,0.406432
32,0.246955,0.251824,0.261320,0.301364,0.240421,0.677847,0.152107,0.650394,0.359361
35,0.329700,0.345059,0.396044,0.403043,0.472854,0.287826,0.304725,0.239381,0.362960
36,0.293999,0.259318,0.282788,0.360842,0.323867,0.329879,0.353300,0.333539,0.604061
37,0.239892,0.220883,0.447163,0.443715,0.374524,0.325597,0.328626,0.408209,0.352983
...,...,...,...,...,...,...,...,...,...
424,0.211562,0.142474,0.785516,0.386134,0.361296,0.342168,0.184937,0.168489,0.559018
425,0.306663,0.238587,0.426167,0.401231,0.506186,0.294095,0.299420,0.296495,0.372749
426,0.314137,0.290121,0.517580,0.374248,0.256840,0.341100,0.333903,0.349291,0.364372
428,0.307663,0.282272,0.377610,0.382645,0.434304,0.299115,0.426110,0.349712,0.282161


In [6]:
tuning_curves_resampled = pd.read_csv(DATA + 'data_tuning-curves_resampled.csv', index_col=0)
tuning_curves_resampled

,3.0,3.6,4.3,5.2,6.3,7.6,9.1,10.9,13.2,15.8,19.0,22.9,27.6,33.2,39.9,48.0
23,16.456894,13.386917,14.241179,25.176488,31.539623,24.015185,21.664833,27.733209,26.075725,14.932467,15.453646,23.010958,21.232581,17.190902,18.695779,24.550303
32,19.666185,19.840686,20.087344,20.425779,21.023485,22.783436,23.997637,20.397993,23.889550,49.052892,45.342350,14.347820,23.870124,50.444565,49.809992,28.617633
35,22.065571,22.414608,23.225313,25.095349,26.715004,26.801151,27.604952,30.739339,30.019477,21.070523,19.405530,20.527198,18.803809,16.194653,17.629603,24.291556
36,23.119366,21.303242,20.403798,21.063983,22.841923,26.785243,28.193449,26.116571,25.504507,25.735888,26.462601,27.534032,27.615674,26.254725,31.578954,47.501961
37,16.372349,14.379730,15.591805,24.712940,30.771667,31.016037,29.484790,26.786906,24.524932,22.618538,21.932779,22.156984,24.246946,27.610574,27.655233,24.090648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,21.538996,12.356139,16.589937,56.295555,78.370200,52.218417,38.760202,37.348634,36.384169,35.540715,31.250490,20.809342,15.684859,16.599502,28.642440,56.913355
425,19.843449,16.181614,15.825217,22.940423,27.695072,26.270272,26.970587,31.579390,30.775813,20.828670,19.111818,19.362033,19.292680,19.180220,20.446969,24.119722
426,23.776134,21.381875,22.509381,33.083372,38.698181,32.618289,26.384500,20.595921,20.495882,25.029322,25.641747,25.223822,25.692074,26.361495,26.978724,27.578304
428,21.905144,20.257191,20.319073,24.231985,27.064660,27.054814,27.817609,30.195631,29.694483,22.591060,23.340457,29.967700,28.661748,25.226558,22.574573,20.089453


# Transpile circuits

- To transpile a pennylane circuit to qasm2 (for qiskit), run it under a quantum tape, then use the qbraid transpiler
- IonQ transpiles to a dictionary which can also be saved for later

In [7]:
def circuit_to_tape(pl_circuit):
    def tape_machine(*args):
        with qml.tape.QuantumTape() as tape:
            pl_circuit(*args)
        return tape

    return tape_machine

In [8]:
tape = {
    'ang': circuit_to_tape(circuit_angle_swap),
    'ang-qft': circuit_to_tape(circuit_angle_qft_swap),
    'amp': circuit_to_tape(circuit_amp_iamp),
    'amp-qft': circuit_to_tape(circuit_amp_iamp_qft),
}

In [9]:
def small_then_big_array(data1, data2):
    if is_array_lesser(data2, data1):
        return data2, data1
    else:
        return data1, data2

In [10]:
def get_fidelity_circuits(datum, tape_machine):
    quantum_circuits = pd.DataFrame(columns=['A', 'B', 'qasm2'])

    # A x B
    pairs, pairs_len = combinations(datum.index, 2), (len(datum) * (len(datum) - 1) // 2)
    for a, b in tqdm(pairs, total=pairs_len):
        a_i, b_i = datum.loc[a].to_numpy(), datum.loc[b].to_numpy()
        a_i, b_i = small_then_big_array(a_i, b_i)
        tape = tape_machine(a_i, b_i)
        circuit = transpile(tape, 'qasm2')
        quantum_circuits.loc[len(quantum_circuits)] = [a, b, circuit]
        # quantum_fidelity.loc[len(quantum_fidelity)] = [b, a, fidelity]

    # A x A (should all be 1)
    for a in tqdm(datum.index):
        a_i = datum.loc[a].to_numpy()
        tape = tape_machine(a_i, a_i)
        circuit = transpile(tape, 'qasm2')
        quantum_circuits.loc[len(quantum_circuits)] = [a, a, circuit]

    return quantum_circuits

In [11]:
get_fidelity_circuits(tuning_curves_rescaled, tape['ang']).to_excel(DATA + 'circuits_ang.xlsx')
get_fidelity_circuits(tuning_curves_rescaled, tape['ang-qft']).to_excel(DATA + 'circuits_ang-qft.xlsx')
get_fidelity_circuits(tuning_curves_resampled, tape['amp']).to_excel(DATA + 'circuits_amp.xlsx')
get_fidelity_circuits(tuning_curves_resampled, tape['amp-qft']).to_excel(DATA + 'circuits_amp-qft.xlsx')

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]